In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# !pip install --upgrade plotly 

In [3]:
# Let's import the libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import seaborn as sns
from pandas_profiling import ProfileReport
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [4]:
# Style des graphiques seaborn
sns.set_theme(style = "whitegrid")

In [5]:
# data_stroke = pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
data_stroke = pd.read_csv("C:/Users/Papa Ba GAYE/Desktop/healthcare-dataset-stroke-data.csv")

In [6]:
data_frame = data_stroke.copy()

# Démarche de travail :
## Objectif mesurable :
- **Objectif** : Prédire si une personne est atteinte d'un accident vasculaire cérébral ou pas à partir des données personnelles et cliniques fournies (problème de classification). Il est essentiel de noter qu'il est plus urgent, dans ce cas de figure, de détecter toutes les personnes malades (et non ceux qui sont vraiment malades parmi ceux qui sont identifiés comme malade). Sinon un patient malade peut être diagnostiquer comme sain alors que ce n'est pas le cas. Nous préférerons ainsi la sensibilité à la précision. 
- **Métrique et score à atteindre** : La proportion de la classe positive est largement inférieure à la proportion de la classe négative (nous notons une forte déséquilibre de classes). Dans ce cas de figure, la métrique accuracy ne sera pas assez performante pour évaluer notre modèle de classification. A la place, nous allons utiliser les métriques sensibilité et précision pour valider notre modèle :
    - Sensibilité : $\frac{VP}{VP+FN}$. Permet de calculer le pourcentage de tests positifs parmi les patients réellement atteints d'AVC ;
    - Précision : $\frac{VP}{VP+FP}$. Permet de calculer le pourcentage de patients réellement malades parmi ceux dont le test est positif ;
    - F1-score : $\frac{2\times Sensibilite \times Precision}{Sensibilite+Precision}$ ;
    - Avec :
        - $VP$ : Nombre de Vrais Positifs ;
        - $FN$ : Nombre de Faux Négatifs ;
        - $FP$ : Nombre de Faux Positifs ;
    
  Nous nous fixons une sensibilité supérieure à **80%**.
## Exploration des données :
### Analyse de la forme :

- **Cible** : La variable dépendante est la variable **stroke** qui contient des données discrètes. Cette variable indique si une personne est atteinte d'un accident vasculaire cérébral (1 pour atteinte) ou pas (0 pour non atteinte). Nous allons abréger, par la suite, accident vasculaire cérébral par AVC pour faciliter l'écrit. Nous devons transformer, plus tard, la variable cible en variable catégorielle non ordinale (cela nous permettra de différencier la catégorie "être atteint(e) d'AVC" de la catégorie "être non atteint(e) d'AVC"). Nous devrons ainsi utiliser un modèle de classification (classification model) pour déterminer si un patient est malade ou sain.

- **Nombre de lignes et de colonnes** : Nous avons identifié 5110 observations et 12 variables (dont la cible). Le nombre d'observations est inférieur à 10000, donc notre dataset ne contient pas beaucoup d'observations mais assez pour entraîner un modèle.
- **Types de variables** (à l'exception de la variable stroke) : Parmi les features, nous avons identifié 7 variables catégorielles dont deux contiennent des valeurs discrètes et les autres contiennent des valeurs de type object (chaîne de caractères) et 3 variables non catégorielles. 

    - Les variables de type object ont pour valeurs possibles les suivantes :
    
        - **gender** (genre) : contient les valeurs *Male* (Homme), *Female* (Femelle) ou *Other* (ni Homme, ni Femme) ;
    
        - **ever_married** (jamais marié(e)) : peut prendre les valeurs *Yes* (Oui) ou *No* (Non) ;
    
        - **work_type** (type de travail) : peut prendre les valeurs *Private* (Privée), *Self_employed* (Auto emploi ou Travail autonome), *Govt_job* (Travail au gouvernement), **children** (enfant), **Never_worked** (N'a jamais travaillé(e)) ;
    
        - **Residence_type** (type de résidence) : peut prendre les valeurs *Urban* (Urbaine), *Rural* (Rurale) ;
    
        - **smoking_status** (statut de fumeur) : contient les valeurs *formerly_smoked* (a fumé(e) dans le passé), *never_smoke* (n'a jamais fumé), *smokes* (fume), *Unknow* (donnée non recueillie).
    
    - Quant aux variables catégorielles **hypertension** et **heart_disease** (maladie de coeur), elles peuvent prendre les valeurs 1 (pour *positive*) ou 0 (pour *négative*). 
    
    - Nous identifions une variable discrète non categorielle (**age**) dont les valeurs varient de 0.08 (enfant de 9 mois) à 82 (adulte de 82 ans). Sa valeur moyenne est de 43 (adulte de 43 ans).
    
    - Le dataset ne contient que deux variables continues, **avg_glucose_level** (niveau moyen de glucose dans le sang) et **bmi** (indice de masse corporelle). Ces deux variables ne s'expriment pas avec les mêmes unités :
    
      - La variable avg_glucose_level a pour valeur maximale 271.74 et pour valeur minimale 55.12. Sa valeur moyenne est de 106.15.
      
      - La variable bmi a pour valeur maximale 97.60 et pour valeur minimale 10.30. Sa valeur moyenne est 28.89.
     
    - Nous définirons plus en détail ces variables dans la partie analyse du fond.
    
- **Identification des valeurs manquantes** : Nous remarquons que seule la variable bmi contient des valeurs manquantes qui sont un peu dispersées dans le dataset (Un peu entassées au niveau des premières observations). Mais en sachant que le nombre de valeurs manquantes ne représente que 16% dans l'ensemble des données présentes dans la colonne bmi et que nous allons choisir un échantillon aléatoire pour l'entraînement du modèle (ainsi que pour l'évaluation) donc on peut supposer que le remplacement des valeurs manquantes par la valeur la plus fréquente dans la variable bmi constitue une bonne stratégie.

- **Identification des observations redondantes** : Les données ne contiennent pas d'observations dupliquées.
### Analyse du fond :

- **Visualisation de la cible** : Nous remarquons 95.13% de personnes non atteintes d'AVC contre seulement 4.87% de personnes atteintes d'AVC. La proportion de patients non malades est largement supérieure au nombre de patients non malades.

- **Compréhension des différentes variables** : 

    - id : La variable id contient des valeurs discrètes et identifie chaque observation de manière unique. Elle n'apporte aucune information supplémentaire et donc n'influence pas le fait qu'une personne soit malade ou non. La variable id doit être supprimée.
    
    - Variables catégorielles :
    
        - Hypertension : Cette variable indique si oui ou non, le patient souffre d'hypertension. Un patient est testé positif à l'hypertension si on constate à deux reprises, et pas dans le même jour, une tension systolique supérieure ou égale à 140 mm Hg et/ou une tension diastolique supérieure ou égale à 90 mm Hg. Selon les tests cliniques, une hypertension peut souvent être la cause d'AVC. Nous verrons par la suite si les données recueillies de cette variable sont fiables. La colonne hypertension contient 90% de tests négatifs et 10% de tests positifs.
        
        - heart_disease : La variable heart_disease indique si le patient est atteint de cardiopathie (maladie cardiaque) ou pas. Il y a différents types de cardiopathies et nous verrons si ces derniers peuvent influencer le risque d'attraper un AVC. La colonne heart_disease contient 95% de tests positifs contre seulement 5% de tests négatifs. Cependant, nous savons que l'hypertension non traitée peut causer la cardiopathie. Donc on peut supposer, d'hors et déja, que ces deux variables sont fortement corrélées (hypothèse à vérifier).
        
        - gender : La variable gender indique à quel sexe appartient le patient. On doit vérifier si le sexe du patient peut influencer le risque qu'il soit atteint d'AVC. La colonne gender est composée de 58% de femmes, de 41% d'hommes et très peu (presque 0%) de sexe de type autres.
        
        - ever_married : Cette variable indique si le patient a déja été marié. L'analyse de cette variable doit nous permettre de dire si le patient a plus de chance d'attraper un AVC en s'étant déja marié (dans le temps présent ou passé) ou pas. Nous notons 66% des patients qui se déclarent ne s'être jamais mariés et 34% qui se sont déja mariés. 
        
        - work_type : Elle indique le type de travail effectué par un patient. Nous avons identifié 57% de patients travaillant dans le secteur privé (professions et secteurs d'activité ne dépendant pas de l'Etat), 16% des patients effectuant du travail autonome (ils sont leurs propres employés), 13% des patients sont des enfants (on n'a pas plus d'informations sur la catégorie children mais on suppose pour l'instant que le patient est un enfant et donc qu'il n'a pas besoin de travailler), presque 13% des patients travaillent pour le gouvernement (dans le secteur public) et seulement 0.4% n'ont jamais travaillé. Pour les patients dont le type de travail est children, nous devons vérifier si leurs ages indiquent que ce sont des enfants ou pas (c'est à dire certains sont des adultes).   
        
        - residence_type : Cette variable indique le type de résidence du patient. 51% des patients résident dans un milieu urbain (ville) et 49% des patients résident dans un milieu rural (campagne). Les proportions de ces deux classes sont presque similaires.
        
        - smoking_type : Elle indique si le patient est/était un fumeur ou pas. 37% des patients n'ont jamais fumés, 30% des patients n'indique pas s'il fument, 17% des patients ont fumé auparavant et 15% des patients disent fumer au moment où on les interrogeait. La catégorie unknow (inconnue) peut constituer un problème car elle n'apporte aucune information utile. Nous vérifierons par la suite si cette variable apporte de l'information à notre modèle.
        
    - variables non catégorielles :
        
        - age : Nous constatons que les patients qui  sont agées entre 37 et 63 ans sont plus nombreux dans la base de données, suivis des patients qui sont agés entre 78 et 82 ans tandis que les plus jeunes sont les moins nombreux.  
        
        - avg_glucose_level : Le niveau moyen de glucose dans le sang indique si, oui ou non, le patient est atteint de diabète. Cette variable s'exprime en mg/dL (milligrammes par déciLitre). Les patients qui souffre de diabète ont un niveau moyen de glucose supérieur ou égale à 200 mg/dL. Par contre, un niveau moyen de glucose inférieur à 140 mg/dL est considéré comme normal. Nous constatons que la plupart des patients n'ont pas de diabète car la plus grande partie des patients ont un niveau moyen de glucose tournant autour de 75-87 mg/dL de sang. Nous remarquons qu'environ 10 à 20% des patients ont un niveau moyen de glucose tournant autour de 210-225 mg/dL. On peut considérer que ces derniers sont atteints de diabète.
        
        - bmi : L'indice de masse corporelle (IMC) permet d'indiquer la corpulence d'un patient. Il s'exprime en kg/m2 (kilogrammes par mètre carré). Une personne est considérée comme obèse si son IMC dépasse 30 kg/m2. La plupart des patients éxaminés ont un IMC situé autour de 29 kg/m2. Ces derniers présentent un cas de surpoids ou d'obésité modérée (dont les IMC sont respectivement situés entre 25 et 30 kg/m2 et entre 30 et 35 kg/m2).
        
- Etude plus poussée des variables avec pandas-profiling : ##
            
- **Visualisation des relations variables_explicatives - variable_a_expliquer** :
    
    - Relations Cible - variables catégorielles : 
        
        - Pour la variable work_type : Nous remarquons que les patients qui ont pour type de travail children ont plus de chance de ne pas attraper d'AVC que les patients qui effectuent d'autres types de travail. On a peu de patients non travailleurs dans la variable work_type donc on ne peut rien dire par rapport à cette catégorie (mais il est possible qu'elle influence aussi le fait qu'on soit atteint d'AVC ou pas). 
        
        - Pour la variable smoking_type : Nous remarquons que les patients ne disant pas s'ils fument/fumaient présentent une proportion de malades inconsistante par rapport aux autres types de fumeurs. Cela est dû au fait que la catégorie unknow n'est pas fiable (elle se comporte comme une valeur manquante).
        
    - Relations Cible - variables non catégorielles :
        
        - Parmi les variables non catégorielles, seule la variable age influence le fait qu'un patient soit atteint d'AVC ou pas. Il y a plus de risque d'attraper un AVC chez les patients plus âgés ;
        
        - Pour les deux autres variables restantes nous constatons que les distributions sont presque les mêmes pour les patients malades et non malades ;
        
        - Nous vérifierons plus amplement ces hypothèses à travers un test de student.
        
- **Visualisation des relations entre variables quantitatives** (corrélations) : Les variables quantitatives ne partagent aucune forte corrélation entre elles.

- **Visualisation des relations entre variables catégorielles et quantitatives** : Nous constatons que quelques variables catégorielles ont de fortes corrélations avec des variables quantitatives.

    - La variable **age** est fortement corrélée avec les variables **work_type**, **smoking_status** et **ever_married**. Pour sa relation avec les autres variables catégorielles nous remarquons de légères corrélations ou quasiment pas de corrélations pour certaines.
    
    - La variable **bmi** est, pour son cas, fortement corrélée avec les variables **ever_married** et **work_type**. 
    
    - En revenant au niveau de pandas profiling, nous constatons que nos analyses sont soutenues par les remarques faites par la librairie (aucune incohérence n'est à noter).
   
- **Identification des valeurs aberrantes** : Tous les trois variables comportent des valeurs aberrantes (anormales).

    - La variable age ne comporte que deux valeurs aberrantes du coté de la classe *test positif* de la variable stroke ;
    
    - Les variables bmi et avg_glucose_level contiennent plusieurs valeurs aberrantes.
    
    - La variable avg_glucose_level ne contient des valeurs aberrantes que du coté de la classe *test negatif* de la variable stroke alors que la variable bmi en comporte pour les deux classes (mais beaucoup plus du coté de la classe *test negatif*)
    
- **Première conclusion** : 

    - On constate une dépendance entre la variable cible et la variable age.  
    
    - La variable id ne comporte aucune information utile donc il est préférable de la supprimer de la base de données.

    - La base de données contient des valeurs manquantes que nous allons remplacer par la valeur la plus fréquente de la variable bmi ;
    
    - La catégorie unknow de la variable smoking_status n'apporte aucune information supplémentaire donc nous devons la remplacer par une catégorie plus intéressante (*never smoked* par exemple) ;
    
    - Les variables quantitatives ne sont pas corrélées entre elles ;
    
    - La variable age est fortement corrélée avec certaines variables catégorielles. Notemment les variables work_type, ever_married et smoking_status ; 
    
    - La variable bmi est aussi fortement corrélée avec les variables catégorielles ever_married et work_type ;
    
    - Nous devons garder les variables age et bmi et supprimer les variables smoking_status, work_type et ever_married ;
    
    - Des tests supplémentaires vont être effectuer pour etayer certaines hypothèses ou les rejeter ;
    
    - Les variables quantitatives comportent des valeurs aberrantes. Nous allons vérifier s'il est nécessaire de les remplacer ou de les supprimer (il serait mieux de les supprimer pour ne pas changer la distribution de nos variables) ;
    
    - Nous allons retenir pour l'instant les variables hypertension et heart_disease qui sont cliniquement très intéressantes pour nos analyses.

- **Tests d'hypothèses** : Ces tests nous permetttrons de valider ou de rejeter certaines hypothèses.

    - Testons si les variables quantitatives influencent le fait qu'un patient soit atteint d'AVC ou pas (nous noterons H0 l'hypothèse selon laquelle une variable n'a pas d'influence sur la cible) :
        
        - Le test de student nous indique que les variables **age** et **avg_glucose_level** **influencent le fait qu'un patient soit atteint d'AVC ou pas**.
        
        - La variable **bmi n'entretient pas de dépendance avec la variable stroke**.

- **Seconde conclusion** : 
    
    - Les variables age et avg_glucose_level sont informatives contrairement à la variable bmi qui elle peut-être supprimée.
        
    - Les variables à supprimer sont donc : bmi, work_type, ever_married, smoking_status, id. Pour le moment nous ne supprimerons que ces variables. Nous ferons une sélection antérieure de variables si on constate un over-fitting sur les données de test.

# Exploration des données

## Identification de la cible

In [7]:
# Vérifions le contenu des dix premières lignes  
data_frame.head(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


Nous savons que la cible est la variable stroke. C'est la variable à expliquer.

In [8]:
# Déterminons le type de la cible et son contenu
# Type de la variable
data_frame["stroke"].dtype

dtype('int64')

In [9]:
# Vérifions les valeurs possibles du target
data_frame["stroke"].unique()

array([1, 0], dtype=int64)

## Nombre de ligne et de colonnes

In [10]:
# Forme du dataframe
data_frame.shape

(5110, 12)

Nous avons 5110 observations et 12 variables (dont la cible)

## Types des variables explicatives

In [11]:
# Identifions les types des variables
data_frame.dtypes

id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

Nous identifions 7 colonnes de type catégorielles. Nous allons stocker les noms de ces colonnes dans une variable.

In [12]:
categorical_columns = ["hypertension", "heart_disease"]
categorical_columns.extend(data_frame.select_dtypes('object').columns)

In [13]:
# Pour chaque variable de type object vérifions ses catégories
for column in data_frame.select_dtypes('object').columns:
    print(f"Valeurs uniques de {column:-<20} {data_frame[column].unique()}\n")

Valeurs uniques de gender-------------- ['Male' 'Female' 'Other']

Valeurs uniques de ever_married-------- ['Yes' 'No']

Valeurs uniques de work_type----------- ['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']

Valeurs uniques de Residence_type------ ['Urban' 'Rural']

Valeurs uniques de smoking_status------ ['formerly smoked' 'never smoked' 'smokes' 'Unknown']



In [14]:
# Vérifions les valeurs que contient les variables hypertension et heart_disease
for column in categorical_columns[:2]:
    print(f"Unique values of {column:-<20} {data_frame[column].unique()}\n")

Unique values of hypertension-------- [0 1]

Unique values of heart_disease------- [1 0]



Stockons dans une autre variable les noms des colonnes non catégorielles

In [15]:
non_categorical_columns = ["age", "avg_glucose_level", "bmi"]

In [16]:
# Vérifions les statistiques des colonnes non catégorielles
data_frame[non_categorical_columns].describe()

,age,avg_glucose_level,bmi
count,5110.000000,5110.000000,4909.000000
mean,43.226614,106.147677,28.893237
std,22.612647,45.283560,7.854067
min,0.080000,55.120000,10.300000
25%,25.000000,77.245000,23.500000
50%,45.000000,91.885000,28.100000
75%,61.000000,114.090000,33.100000
max,82.000000,271.740000,97.600000


## Identification des valeurs manquantes

Première analyse des valeurs manquantes par sommation

In [17]:
data_frame.isna().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [18]:
# Stockage du dataframe des données manquantes dans une variable
na_values = data_frame.isna()

Deuxième analyse par visualisation (Visualisation de la répartition des valeurs manquantes dans le dataset)

In [19]:
# px.imshow(na_values)

Nous remarquons que seule la variable bmi contient des valeurs manquantes qui sont un peu dispersées dans le dataset (Un peu entassées au niveau des premières observations). 

Vérifions le pourcentage de valeurs manquantes

In [20]:
percentage_of_na = na_values["bmi"].sum()/data_frame.shape[1]

In [21]:
print(f"La variable bmi contient {percentage_of_na}% de valeurs manquantes.")

La variable bmi contient 16.75% de valeurs manquantes.


## Identification des observations redondantes

In [22]:
# Vérifions s'il y a des observations dupliquées dans le dataset
data_frame.duplicated().sum()

0

Le dataset ne contient pas d'observations dupliquées.

## Visualisation de la cible

Nous devons voir la proportion de chaque classe au sein de la variable cible pour identifier si on a affaire à un désiquilibre de classe.

In [23]:
# Vérifions la proportion de chaque classe
proportions = data_frame["stroke"].value_counts(normalize = True)*100
proportions

0    95.127202
1     4.872798
Name: stroke, dtype: float64

Visualisons la proportion de chaque classe à l'aide d'un diagramme en camembert

In [24]:
proportions.plot.pie()

<AxesSubplot:ylabel='stroke'>

Nous remarquons que 95.13% des personnes sont atteintes d'AVC contre seulement 4.87% de personnes atteintes d'AVC.

In [25]:
# Transformation en variable catégorielle de la cible
data_frame["stroke"] = data_frame["stroke"].astype("category")

## Compréhension des différentes variables

Analysons les variables à l'aide de graphiques.
Nous devons d'abord transformer les types ('object' et 'int') des variables catégorielles en type 'category' pour faciliter les analyses.

In [26]:
# Transformation en type category de tous les variables catégorielles
for column in categorical_columns:
    data_frame[column] = data_frame[column].astype('category')

In [27]:
data_frame[categorical_columns].dtypes

hypertension      category
heart_disease     category
gender            category
ever_married      category
work_type         category
Residence_type    category
smoking_status    category
dtype: object

### Variables catégorielles

Identifions la proportion de chaque classe.

In [28]:
# Récupérons les proportions des variables dans une liste
proportions = []
for column in categorical_columns:
    proportion = data_frame[column].value_counts(normalize = True)*100
    proportions.append(proportion)
    print(f"Colonne {column} :\n{proportion}\n-------------------\n")

Colonne hypertension :
0    90.254403
1     9.745597
Name: hypertension, dtype: float64
-------------------

Colonne heart_disease :
0    94.598826
1     5.401174
Name: heart_disease, dtype: float64
-------------------

Colonne gender :
Female    58.590998
Male      41.389432
Other      0.019569
Name: gender, dtype: float64
-------------------

Colonne ever_married :
Yes    65.616438
No     34.383562
Name: ever_married, dtype: float64
-------------------

Colonne work_type :
Private          57.240705
Self-employed    16.027397
children         13.444227
Govt_job         12.857143
Never_worked      0.430528
Name: work_type, dtype: float64
-------------------

Colonne Residence_type :
Urban    50.802348
Rural    49.197652
Name: Residence_type, dtype: float64
-------------------

Colonne smoking_status :
never smoked       37.025440
Unknown            30.215264
formerly smoked    17.318982
smokes             15.440313
Name: smoking_status, dtype: float64
-------------------



In [29]:
# Tracons le diagramme en barres des proportions pour chaque variable
fig, axs = plt.subplots(4, 2, figsize = (16, 14))
axs = axs.flat

for i, p in enumerate(proportions):
    fig.tight_layout(pad = 3)
    sns.barplot(x = p.index, y = p.values, ax = axs[i])
    axs[i].set_title(f"Diagramme en barres {p.name}")
    axs[i].set_xlabel(p.name)
    axs[i].set_ylabel("Pourcentages de valeurs")
    
fig.delaxes(axs[7])
    

### Variables non catégorielles

Comptons le nombre de patients par age à l'aide d'un countplot.

In [30]:
plt.figure(figsize = (17, 17))
sns.countplot(data = data_frame, y = "age", palette = "viridis")
plt.title("Nombre de patients par age")

Text(0.5, 1.0, 'Nombre de patients par age')

Nous constatons qu'il y a plus de patients adultes que de patients jeunes. Le plus grand nombre de tests a été effectué sur les personnes agées de 78 ans. Visualisons la distribution de la variable age. 

In [31]:
plt.figure(figsize = (15, 8))
sns.histplot(data = data_frame, x = "age", kde = True)
plt.title("Distribution de la variable age")

Text(0.5, 1.0, 'Distribution de la variable age')

La variable age ne suit pas une distribution normale. Nous remarquons que la distribution a tendance à être plus élevée au niveau des personnes agées entre 37 et 63 ans. La distribution est plus basse au niveau des jeunes patients.

Tracons les histogrammes et les densitées du niveau moyen de glucoses dans le sang et de l'indice de masse corporelle.

In [32]:
fig, axs = plt.subplots(1, 2, figsize = (17, 6))

axs = axs.flat

for i, column in enumerate(non_categorical_columns[-2:]):
    fig.tight_layout(w_pad = 3, pad = 1.2)
    color = "green" if i == 0 else "blue" 
    sns.histplot(data = data_frame, x = column, kde = True, ax = axs[i], color = color)
    

Nous remarquons, pour la variable avg_glucose_level, une composition de deux distributions (la deuxième à l'aire de se détacher de la première). La première distribution ressemble à une distribution normale avec une plus grande variance que la deuxième et sa moyenne tourne autour de 75-87 mg/dL de sang. Pour la deuxième distribution, dont la variance est plus faible, nous notons une moyenne tournant autour de 210-225 mg/dL.  

Nous remarquons, pour la variable bmi, une distribution qui semble être normale avec une moyenne tournant autour de 30 kg/m2.

## Utilisation de Pandas Profiling

Utilisons la librairie pandas profiling pour affiner nos analyses

In [33]:
profile = ProfileReport(data_frame, title = "Investigation avec Pandas Profiling")

In [34]:
# profile

Nous reviendrons à quelques remarques effectuées par pandas profiling plus tard. Nous pouvons passer à l'analyse des relations entre les variables.

## Relations entre la variable cible et les variables explicatives

### Variables catégorielles / variable cible

Analysons ces relations en utilisant les tableaux croisés (de comptage) entre les variables.

In [35]:
# Affichons des tableaux croisés entre la cible et chaque variable catégorielle à l'aide d'un heatmap.

fig, axs = plt.subplots(4, 2, figsize = (20, 16), sharey = True)
axs = axs.flat

for i, column in enumerate(categorical_columns):
    fig.tight_layout(pad = 3, h_pad = 4)
    sns.heatmap(pd.crosstab(data_frame["stroke"], data_frame[column]), annot = True, fmt = "d", ax = axs[i])
    axs[i].set_title(f"Tableau croisé entre stroke et {column}", fontsize = 17)
    axs[i].set_xlabel(column, fontsize = 14)
    
fig.delaxes(axs[7])

Nous remarquons que pour la variable work_type, on a une proportion de patients malades moins importante au niveau de la catégorie children.
Nous remarquons également que pour la variable smoking_status, la proportion de patients malades est moins importante au niveau de la catégorie unknown. Pour le reste des variables on ne peut rien dire pour l'instant.

### Variables non catégorielles / variable cible

Dans cette partie, nous devons affiner les analyses effectuées sur les variables non catégorielles en tracant leurs distributions suivant les différentes classes possibles de la variable cible. Nous verifierons ensuite si les distributions obtenues varient de la même manière. 

**Pour la variable age**

In [36]:
plt.figure(figsize = (15, 6))
sns.histplot(data = data_frame, x = "age", hue = "stroke", kde = True, palette = "tab10")
plt.title(f"Histogramme de la variable age", fontsize = 14)
plt.xlabel("age", fontsize = 13)
plt.ylabel("stroke", fontsize = 12)
plt.tight_layout(pad = 5)

Nous voyons que les deux distributions sont un peu différentes. Nous constatons que les personnes plus agées ont plus de risque d'être atteintes d'AVC.

**Pour la variable avg_glucose_level**

In [37]:
plt.figure(figsize = (15, 8))
sns.histplot(data = data_frame, x = "avg_glucose_level", hue = "stroke", kde = True, palette = "tab10")
plt.title(f"Histogramme de la variable avg_glucose_level", fontsize = 14)
plt.xlabel("avg_glucose_level", fontsize = 13)
plt.ylabel("stroke", fontsize = 12)
plt.tight_layout(pad = 5)

Nous ne remarquons aucune différence entre les deux distributions.

**Pour la variable bmi**. Remplacons les valeurs manquantes par le mode pour voir les distributions qu'on obtient.

In [38]:
mode = data_frame["bmi"].mode()
data_to_plot = data_frame.fillna(mode)
plt.figure(figsize = (15, 8))
sns.histplot(data = data_to_plot, x = "bmi", hue = "stroke", kde = True, palette = "tab10")
plt.title(f"Histogramme de la variable bmi", fontsize = 14)
plt.xlabel("bmi", fontsize = 13)
plt.ylabel("stroke", fontsize = 12)
plt.tight_layout(pad = 5)

Nous obtenons les mêmes distributions à première vue (il y a une nette différence mais pas très visible entre les deux distributions).

## Relations entre variables non catégorielles

Vérifions si certaines variables non catégorielles ont une forte corrélation entre elles.

In [39]:
# px.imshow(data_frame[non_categorical_columns].corr().round(2))

Nous ne remarquons qu'aucune des coefficients de corrélation obtenues ne dépasse 0.5%. Les variables quantitatives ne partagent aucune corrélation entre elles.

## Relations entre variables catégorielles et non catégorielles

Nous allons vérifier pour chaque variable quantitative si elle est influencée par une ou plusieurs variable(s) catégorielle(s).

In [40]:
def rel_cat_quant(categorical_column):
    """Fonction pour vérifier les relations pouvant exister entre les variables qualitatives
    et quantitatives. Nous allons utiliser la base de données ne comportant pas de valeurs 
    manquantes.
    
    Args:
        categorical_column(str): Nom de la variable catégorielle
    
    Returns:
        None
    """
    
    fig, axs = plt.subplots(2, 2, figsize = (18, 13))

    axs = axs.flat

    for i,column in enumerate(non_categorical_columns):

        sns.histplot(data = data_to_plot, x = column, kde = True, hue = categorical_column, ax = axs[i], palette = "tab10")

        axs[i].set_title(f"Variable {column}", fontsize = 14)

    fig.delaxes(axs[3])

    fig.tight_layout(pad = 3)

**Pour la variable hypertension**

In [41]:
rel_cat_quant("hypertension")

Nous remarquons une **légère dépendance** entre les variables **hypertension** et **age**. 

**Pour la variable heart_disease**

In [42]:
rel_cat_quant("heart_disease")

Nous constatons également une **légère dépendance** entre les variables **heart_disease** et **age**.

**Pour la variable gender**

In [43]:
rel_cat_quant("gender")

Nous remarquons une **légère corrélation** (distributions un peu différentes) entre la variable **age** et **gender**.

**Pour la variable ever_married**

In [44]:
rel_cat_quant("ever_married")

La variable **ever_married** est **fortement corrélée** avec les variables **age** et **bmi**.

**Pour la variable work_type**

In [45]:
rel_cat_quant("work_type")

La variable **work_type** également réalise une **forte corrélation** entre les variables **age** et **bmi**. *Nous remarquons que la catégorie children de work_type désigne les patients qui sont des enfants (donc la variable work_type est cohérente pour le reste de l'analyse)*.

**Pour la variable residence_type**

In [46]:
rel_cat_quant("Residence_type")

La variable **Residence_type** est **légèrement corrélée** (on peut même dire qu'elles ne sont pas corrélées) avec la variable **age**.

**Pour la variable smoking_status**. Remplacons la catégorie *unknown* par *never smoked* pour avoir une meilleure représentation de la variable.

In [47]:
data_to_plot["smoking_status"] = data_to_plot["smoking_status"].apply(lambda x: "never smoked" if x == "Unknown" else x)

In [48]:
data_to_plot["smoking_status"].value_counts(normalize=True)*100

never smoked       67.240705
formerly smoked    17.318982
smokes             15.440313
Name: smoking_status, dtype: float64

In [49]:
rel_cat_quant("smoking_status")

La variable **smoking_status** est **fortement corrélée** avec la variable **age**.

## Identification des valeurs aberrantes

Tracons les boxplots des variables catégorielles en les séparant par classe de patients (atteints d'AVC ou pas).

In [50]:
fig, axs = plt.subplots(2, 2, figsize = (18, 14))

axs = axs.flat

for i,column in enumerate(non_categorical_columns):

    sns.boxplot(data = data_to_plot, x = "stroke", y = column, hue = "stroke", ax = axs[i], palette = "tab10")

    axs[i].set_title(f"Variable {column}", fontsize = 14)

fig.delaxes(axs[3])

fig.tight_layout(pad = 3)

Nous constatons que tous les trois variables comportent des valeurs aberrantes. Surtout du coté de la classe *test negatif* de la variable stroke (pour les variables bmi et avg_glucose_level).

## Test d'hypothéses

Vérifions si certaines variables quantitatives influencent le fait qu'un patient soit atteint d'AVC ou pas avec un test de student.

In [51]:
# Définition de la fonction pour effectuer des tests
def student_test(dataframe1: pd.DataFrame, dataframe2: pd.DataFrame, colonnes: list, alpha: float):
    """Cette fonction permet d'effectuer un test de student.
    
    Args:
        dataframe1(pandas.DataFrame): Les données de la première classe.
        dataframe2(pandas.DataFrame): Les données de la deuxième classe.
        colonne(list): Les noms des colonnes sur les quelles on va réaliser le test.
        alpha(float): Le taux d'erreur alpha de l'hypothèse nulle.
    
    Returns:
        None
    """
    
    # Choix d'un échantillon pour le dataframe qui a la plus grande taille
    if dataframe1.shape[0] <= dataframe2.shape[0]:
        dataframe2 = dataframe2.sample(dataframe1.shape[0])
    else:
        dataframe1 = dataframe1.sample(dataframe2.shape[0])
        
    for colonne in colonnes:
        stat, p = ttest_ind(dataframe1[colonne], dataframe2[colonne])
        if  p < alpha:
            print(f"{colonne:-<30} : H0 rejetée")
        else:
            print(f"{colonne:-<30} : H0 non rejetée")

Nous devons d'abord séparer le dataframe entre les catégories *test positif* et *test negatif*

In [52]:
df_positif, df_negatif = data_frame[data_frame["stroke"] == 1], data_frame[data_frame["stroke"] == 0]

In [ ]:
# Effectuons le test
student_test(df_positif, df_negatif, non_categorical_columns, 0.02)

age--------------------------- : H0 rejetée
avg_glucose_level------------- : H0 rejetée
bmi--------------------------- : H0 non rejetée


Selon le test **seule la variable bmi n'influence pas le fait qu'un patient soit malade ou pas**.

# Phase de traitement des données

## Étapes du traitement
Nous allons effectuer plusieurs traitements et vérifier le score obtenu à l'aide d'un arbre de décision (il ne s'agit pas du modèle final). L'arbre de décision est le modèle de classification le plus facile à interpréter c'est pourquoi il est nécessaire de l'utiliser durant la phase de traitement.
- **Suppression de quelques variables** : 
- **Suppression des données aberrantes** : Toutes les valeurs de la variable avg_glucose_level supérieures à environ 169 mg/dL, soit les valeurs désignant les patients qui ont un taux anormal de glucose dans le sang, sont supprimées. Nous obtenons ainsi une distribution à peu près normale pour la variable avg_glucose_level. La distribution de la variable age reste à peu près la même. Après la suppression des valeurs aberrantes, la taille de l'échantillon est de **4483**. Ce sera suffisant pour entraîner le modèle.
- **Encodage des données qualitatives** : 
- **Séparation des données** en données d'entraînement et de test : Les données d'entraînement contiennent **3586** observations et les données de test contiennent **897** observations.
- **Entraînement du modèle de test** : 
- **Evaluation du modèle** : En supprimant uniquement les variables bmi, work_type, ever_married, smoking_status et id on obtient une sensibilité de **96%** pour la classe *test negatif* et seulement **20%** pour la class *test positif*.
- **Selection de variables** : Les variables gender, hypertension, heart_disease et Residence_type sont celles qui doivent être supprimées parce que n'apportant assez d'information par rapport aux variables age et avg_glucose_level.
- **Entraînement et évaluation du modèle de test** avec les variables choisies : Nous obtenons une sensibilité de **96%** pour la classe *test negatif* et **29%** (soit une amélioration de 9%) pour la classe *test positif*.


Conception d'une classe comprenant les différentes étapes du traitement.

In [ ]:
class Traitement:
    """Cette classe va nous permettre d'effectuer des traitements sur les données.
    
    Attributes:
        dataframe(pandas.DataFrame): Les données avant tout traitement.
        cible(str): Le nom de la variable cible
    """
    
    def __init__(self, dataframe, cible):
        """Initialisation des attributs
        """
        if cible in dataframe.columns:
            self.dataframe = dataframe.copy()
            self.cible = cible
        else:
            raise ValueError("Le nom de la cible ne correspond à aucune colonne")
    
    def colonne_existe(self, colonne):
        """Fonction permettant de vérifier si une colonne existe.
        """
        return True if colonne in self.dataframe.columns else False
    
    def supprimer_colonnes(self, colonnes):
        """Cette fonction permet de supprimer des colonnes.
        """
        for colonne in colonnes:
            if self.colonne_existe(colonne):
                self.dataframe.drop(colonne, axis = 1, inplace = True)
            else:
                raise ValueError(f"La colonne {colonne} n'existe pas.")
    
    def modification_categorie(self, colonne, categorie_a_modif, categorie_de_modif):
        """Cette fonction permet de modifier une catégorie par une autre catégorie au sein
        d'une variable catégorielle.
        """
        modif = lambda x: categorie_de_modif if x == categorie_a_modif else x
        if self.colonne_existe(colonne):
            if self.dataframe[colonne].dtype == "category":
                self.dataframe[colonne] = self.dataframe[colonne].apply(modif)
                self.dataframe[colonne] = self.dataframe[colonne].astype('category')
            else:
                raise TypeError(f"La colonne {colonne} n'est pas une colonne catégorielle.")
        else:
            raise ValueError(f"La colonne {colonne} n'existe pas.")
    
    def supprimer_aberrantes(self, colonne):
        """Cette fonction permet de remplacer les valeurs aberrantes par la médiane 
        des données suivant une colonne à préciser
        """
        if self.colonne_existe(colonne):
            quantile_1 = self.dataframe[colonne].quantile(0.25)
            quantile_2 = self.dataframe[colonne].quantile(0.75)
            mediane = self.dataframe[colonne].median()
            inter_quantile = quantile_2 - quantile_1
            limite_bas = quantile_1 - 1.5 * inter_quantile
            limite_haut = quantile_2 + 1.5 * inter_quantile
            # remplacer_aberrante = lambda x : mediane if (x < limite_bas or x > limite_haut) else x
            # self.dataframe[colonne] = self.dataframe[colonne].apply(remplacer_aberrante)
            self.dataframe = self.dataframe[(self.dataframe[colonne] > limite_bas) & (self.dataframe[colonne] < limite_haut)]
        else:
            raise ValueError(f"La colonne {colonne} n'existe pas.")
    
    def encodage(self):
        """Cette fonction permet d'encoder les valeurs qualitatives
        """
        dataframe = self.dataframe.copy()
        for colonne in dataframe.select_dtypes("category").columns:
            dataframe[colonne] = self.dataframe[colonne].cat.codes
        return dataframe
        
    def separer_donnees(self, dataframe):
        """Cette fonction permet de séparer les données d'entraînement des données de test
        """
        y = dataframe[self.cible]
        
        X = dataframe.drop(self.cible, axis = 1)
        
        # from sklearn.preprocessing import PolynomialFeatures
        # polynomial_converter = PolynomialFeatures(degree=2, include_bias=False)
        # polynomial_converter.fit(X)
        # X = polynomial_converter.transform(X)
        
        # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

        from sklearn.model_selection import StratifiedShuffleSplit

        sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=4)
        
        for train, test in sss.split(X, y):
            X_train , X_test = X.iloc[train], X.iloc[test]
            y_train , y_test = y.iloc[train], y.iloc[test]
        
        return X_train, X_test, y_train, y_test
    
    def selection_variables(self, modele, X_train, y_train):
        """Cette fonction permet de sélectionner les variables qui apportent le plus d'information
        au modèle donné en paramètre.
        """
        from sklearn.feature_selection import SelectFromModel
        selector = SelectFromModel(modele)
        selector.fit(X_train, y_train)
        print(f"Colonnes choisies : \n", X_train.columns[selector.get_support()])
        return selector
    
    def premier_entrainement(self, X_train, y_train):
        """Cette fonction permet d'entraîner le premier modèle pour vérifier si le traitement effectué
        est bon.
        """
        from sklearn.tree import DecisionTreeClassifier
        from sklearn.linear_model import LogisticRegression
        
        # modele = DecisionTreeClassifier(random_state = 4)
        modele = LogisticRegression()
        
        modele.fit(X_train, y_train)
        
        return modele
    
    def evaluation(self, modele, X_train, X_test, y_train, y_test):
        """Cette fonction permet d'évaluer un modèle
        """
        from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
        from sklearn.model_selection import learning_curve
        import numpy as np
        
        y_pred = modele.predict(X_test)
        
        print(f"La precision du modèle : {accuracy_score(y_test, y_pred)}")
        print(f"Matrice de confusion : \n{confusion_matrix(y_test, y_pred)}")
        print(f"Rapport de classification : \n{classification_report(y_test, y_pred)}")
        
        N, train_score, test_score = learning_curve(modele, X_train, y_train, cv = 5, train_sizes = np.linspace(0.1, 1, 10))
        
        plt.figure(figsize = (13, 14))
        plt.plot(N, train_score.mean(axis = 1), label = "score_entrainement")
        plt.plot(N, test_score.mean(axis = 1), label = "score_test")
        plt.title(f"Evaluation du modèle {modele.__class__.__name__}")
        plt.legend()

In [ ]:
# Création d'une instance de la classe Traitement 
traitement = Traitement(data_frame, "stroke")

## Première étape : suppression de quelques variables

Supprimons les colonnes bmi et id.

In [ ]:
traitement.supprimer_colonnes(['bmi', 'work_type', 'ever_married', 'smoking_status', 'id'])    

In [ ]:
# Vérifions les variables qui restent
# traitement.dataframe.columns

## Deuxième étape : Suppression des données aberrantes

Effectuons directement la suppression des valeurs aberrantes sans séparer par catégorie.

In [ ]:
traitement.supprimer_aberrantes("age")
traitement.supprimer_aberrantes("avg_glucose_level")

Vérifions les distributions des deux variables quantitatives.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (17, 6))

axs = axs.flat

for i, colonne in enumerate(["age", "avg_glucose_level"]):
    fig.tight_layout(w_pad = 3, pad = 1.2)
    sns.histplot(data = traitement.dataframe, x = colonne, hue = "stroke", kde = True, ax = axs[i], palette = "tab10")
    axs[i].set_title(f"distribution de la colonne {colonne}")

In [ ]:
# Vérifions combien de données il nous reste
traitement.dataframe.shape[0]

4483

## Troisième étape : Encodage des données qualitatives

Nous devons récupérer dans une nouvelle variable le nouveau dataframe dont les données qualitatives seront encodées.

In [ ]:
new_data_frame = traitement.encodage()

In [ ]:
new_data_frame

,gender,age,hypertension,heart_disease,Residence_type,avg_glucose_level,stroke
2,1,80.0,0,1,0,105.92,1
6,1,74.0,1,1,0,70.09,1
7,0,69.0,0,0,1,94.39,1
8,0,59.0,0,0,0,76.15,1
9,0,78.0,0,0,1,58.57,1
...,...,...,...,...,...,...,...
5105,0,80.0,1,0,1,83.75,0
5106,0,81.0,0,0,1,125.20,0
5107,0,35.0,0,0,0,82.99,0
5108,1,51.0,0,0,0,166.29,0


In [ ]:
new_data_frame.isna().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
Residence_type       0
avg_glucose_level    0
stroke               0
dtype: int64

## Quatrième étape : Séparation des données

In [ ]:
X_train, X_test, y_train, y_test = traitement.separer_donnees(new_data_frame)

In [ ]:
# Verifions les tailles des variables explicatives
X_train.shape, X_test.shape

((3586, 6), (897, 6))

## Cinquième étape : Premier entraînement avec une arbre de décision

In [ ]:
modele = traitement.premier_entrainement(X_train, y_train)

C:\Users\Papa Ba GAYE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



## Sixième étape : Evaluation du modèle

In [ ]:
traitement.evaluation(modele, X_train, X_test, y_train, y_test)

La precision du modèle : 0.9632107023411371
Matrice de confusion : 
[[864   0]
 [ 33   0]]
Rapport de classification : 
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       864
           1       0.00      0.00      0.00        33

    accuracy                           0.96       897
   macro avg       0.48      0.50      0.49       897
weighted avg       0.93      0.96      0.95       897



C:\Users\Papa Ba GAYE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Papa Ba GAYE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Papa Ba GAYE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Papa Ba GAYE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logis

## Septième étape : Selection de variables

Vérifions quelles sont les variables qui apportent le plus d'information au modèle d'arbre de décision.

In [ ]:
selector = traitement.selection_variables(DecisionTreeClassifier(random_state = 4), X_train, y_train)

Colonnes choisies : 
 Index(['age', 'avg_glucose_level'], dtype='object')


Les variables qui n'apportent pas d'information au modèle sont les variables gender, hypertension, heart_disease et Residence_type qui vont être supprimées des données d'entraînement et de test

In [ ]:
X_train, X_test = selector.transform(X_train), selector.transform(X_test)

## Huitième étape : Nouveau entraînement du modèle avec les variables choisies

**Entraînement**

In [ ]:
modele = traitement.premier_entrainement(X_train, y_train)

**Evaluation**

In [ ]:
traitement.evaluation(modele, X_train, X_test, y_train, y_test)

La precision du modèle : 0.9632107023411371
Matrice de confusion : 
[[864   0]
 [ 33   0]]
Rapport de classification : 
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       864
           1       0.00      0.00      0.00        33

    accuracy                           0.96       897
   macro avg       0.48      0.50      0.49       897
weighted avg       0.93      0.96      0.95       897



C:\Users\Papa Ba GAYE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Papa Ba GAYE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Papa Ba GAYE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

